In [2]:
# Imports libraries
import requests
import time
import pandas as pd

In [ ]:
# retrieves json files from subreddits 
url_1 = 'https://www.reddit.com/r/movies.json'
url_2 = 'https://www.reddit.com/r/television.json'

In [ ]:
# sends a get request to the urls
res_1 = requests.get(url_1, headers=headers)
res_2 = requests.get(url_2, headers=headers)

In [ ]:
# checks if the get request went through. 
res_1.status_code

In [ ]:
json_file_1 = res_1.json()
json_file_2 = res_2.json()

In [ ]:
# iterates over the json file containing the data from the movie subreddit
[post['data']['name'] for post in json_file_1['data']['children']]

In [ ]:
# iterates over the json file containing the data from the tv subreddit
[post['data']['name'] for post in json_file_2['data']['children']]

In [4]:
# creates a library to pass in as a parameter
headers = {'User-agent': 'fbi surveillance'}

In [5]:
# creates an empty list
posts_m = []

# sets the starting point (the user) to collect the posts 
after = None

for i in range(10):
    print(i)
    if not after:
        params_1 = {'limit': 100}
    else:
        params_1 = {'after': after, 'limit': 100}
    url_1 = 'https://www.reddit.com/r/movies.json'
    res_1 = requests.get(url_1, params = params_1, headers=headers)
    if res_1.status_code == 200:
        json_file = res_1.json()
        posts_m.extend(json_file['data']['children'])
        after = json_file['data']['after']
    else:
        print(res_1.status_code)
        break
    time.sleep(1) # delays execution for 1 sec

0
1
2
3
4
5
6
7
8
9


In [6]:
posts_t = []
after = None
for i in range(10):
    print(i)
    if not after:
        params_2 = {'limit':100}
    else:
        params_2 = {'after': after, 'limit':100}
    url_2 = 'https://www.reddit.com/r/television.json'
    res_2 = requests.get(url_2, params = params_2, headers=headers)
    if res_2.status_code == 200:
        json_file_2 = res_2.json()
        posts_t.extend(json_file_2['data']['children'])
        after = json_file_2['data']['after']
    else:
        print(res_2.status_code)
        break
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9


In [7]:
movie_posts = []
for i in range(812):
    list = {}
    list['post'] = posts_m[i]['data']['selftext']
    list['num_of_comments'] = posts_m[i]['data']['num_comments']
    list['title'] = posts_m[i]['data']['title']
    list['subreddit'] = posts_m[i]['data']['subreddit']
    movie_posts.append(list)
    
movie_posts
df_m = pd.DataFrame(movie_posts)

In [8]:
tv_posts = []
for i in range(922):
    list = {}
    list['post'] = posts_t[i]['data']['selftext']
    list['num_of_comments'] = posts_t[i]['data']['num_comments']
    list['title'] = posts_t[i]['data']['title']
    list['subreddit'] = posts_t[i]['data']['subreddit']
    tv_posts.append(list)
    
tv_posts
df_t = pd.DataFrame(tv_posts)

In [9]:
df_m.head()

,num_of_comments,post,subreddit,title
0,3,[Pet Sematary (2019)](https://www.reddit.com/r...,movies,Official Discussion Weekly Megathread (Pet Sem...
1,106,We're [Jordan Downey](https://www.jordan-downe...,movies,We made a fantasy horror movie with a 30K budg...
2,1249,,movies,"Twenty years ago, an upstart animator named Mi..."
3,921,,movies,"Kumail Nanjiani to star in 'Any Person, Living..."
4,41,,movies,Some Princess Mononoke fan art :)


### Feature Engineering

In [10]:
# combines the `post` column with the `title` column
# into a new column called `posts_combined`
if type(df_m['post']) ==str:
    df_m['posts_combined']= df_m['post'] +'\n'+ df_m['title'] # where posts is not empty
    
else:
    df_m['posts_combined']= df_m['title'] # where post is empty

In [11]:
#repeats the above operation for the tv subreddits
if type(df_t['post']) ==str:
    df_t['posts_combined']= df_t['post'] +'\n'+ df_t['title']
    
else:
    df_t['posts_combined']= df_t['title']

In [12]:
df_3 = [df_m, df_t]

In [13]:
# stacks the movie and tx dataframes
df = pd.concat(df_3, sort=False).reset_index(drop=True)

In [14]:
df.head()

,num_of_comments,post,subreddit,title,posts_combined
0,3,[Pet Sematary (2019)](https://www.reddit.com/r...,movies,Official Discussion Weekly Megathread (Pet Sem...,Official Discussion Weekly Megathread (Pet Sem...
1,106,We're [Jordan Downey](https://www.jordan-downe...,movies,We made a fantasy horror movie with a 30K budg...,We made a fantasy horror movie with a 30K budg...
2,1249,,movies,"Twenty years ago, an upstart animator named Mi...","Twenty years ago, an upstart animator named Mi..."
3,921,,movies,"Kumail Nanjiani to star in 'Any Person, Living...","Kumail Nanjiani to star in 'Any Person, Living..."
4,41,,movies,Some Princess Mononoke fan art :),Some Princess Mononoke fan art :)


In [15]:
reddit_3 = df[['posts_combined', 'subreddit']]

In [16]:
reddit_3['is_tv'] = reddit_3['subreddit'].map({'movies': 0, 'television': 1})

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
reddit_3.tail()

,posts_combined,subreddit,is_tv
1729,Arrested Development - Lucile is in denial,television,1
1730,David Blaine Under NYPD Investigation Over Sex...,television,1
1731,Amanda Bears and Ed ONeil interviewed separate...,television,1
1732,"[Full Frontal with Samantha Bee] S04E07 - ""Apr...",television,1
1733,"""The OA is strange... We’re treated to unflinc...",television,1


In [19]:
reddit_3.to_csv('reddit_post_3.csv', index=False)

In [39]:
# imports previous datasets collected
posts_1 = pd.read_csv('reddit_post_3.csv')
posts_2 = pd.read_csv('reddit_post_train.csv')
posts_3 = pd.read_csv('reddit_posts.csv')

In [40]:
posts_1.head()

,posts_combined,subreddit,is_tv
0,Official Discussion Weekly Megathread (Pet Sem...,movies,0
1,We made a fantasy horror movie with a 30K budg...,movies,0
2,"Twenty years ago, an upstart animator named Mi...",movies,0
3,"Kumail Nanjiani to star in 'Any Person, Living...",movies,0
4,Some Princess Mononoke fan art :),movies,0


In [57]:
posts_1.tail()

,posts_combined,subreddit,is_tv
1538,'Crime Story' Set the Stage for the Last 3 Dec...,television,1
1539,Welcome to the Private Police | A Bit of Fry a...,television,1
1540,Baptiste 1x2,television,1
1541,PBS starts same-time streaming in April as OTT...,television,1
1542,Review w/Forrest MacNeil - Color Copier,television,1


In [41]:
posts_2.head()

,posts_combined,subreddit,is_tv
0,Poster for “Joker” with Joaquin Phoenix,movies,0
1,Detective Pikachu (2019) | Official Chinese Po...,movies,0
2,Lindsay Ellis' video essay on the making of Pe...,movies,0
3,'Superbad' to the Oscars: Jonah Hill's evoluti...,movies,0
4,"Extremely Wicked, Shockingly Evil and Vile | O...",movies,0


In [58]:
posts_2.tail()

,posts_combined,subreddit,is_tv
1528,"With Fox Deal Closed, Disney Now Has TV Studio...",television,1
1529,Just started watching Doom Patrol and i am lov...,television,1
1530,"All My Love for ""Love Death + Robots""",television,1
1531,How Netflix Uses Technology: A Look Behind the...,television,1
1532,Netflix is experimenting with different episod...,television,1


In [42]:
posts_3.head()

,num_of_comments,post,subreddit,title,posts_combined
0,36,Hey Reddit! Trevor Stevens here to talk about ...,movies,"Hi, I’m Trevor Stevens. Director of the film, ...","Hi, I’m Trevor Stevens. Director of the film, ..."
1,1231,NaN,movies,David Harbour Cast In Scarlett Johansson’s ‘Bl...,David Harbour Cast In Scarlett Johansson’s ‘Bl...
2,8457,NaN,movies,JOKER - Teaser Trailer - In Theaters October 4,JOKER - Teaser Trailer - In Theaters October 4
3,31,NaN,movies,Guillermo del Toro Expands the World of ‘Pan’s...,Guillermo del Toro Expands the World of ‘Pan’s...
4,181,NaN,movies,Walt Disney Studios 2019 Release Schedule Unve...,Walt Disney Studios 2019 Release Schedule Unve...


In [47]:
posts_4 = posts_3[['subreddit', 'posts_combined']]

In [59]:
posts_4.head()

,subreddit,posts_combined
0,movies,"Hi, I’m Trevor Stevens. Director of the film, ..."
1,movies,David Harbour Cast In Scarlett Johansson’s ‘Bl...
2,movies,JOKER - Teaser Trailer - In Theaters October 4
3,movies,Guillermo del Toro Expands the World of ‘Pan’s...
4,movies,Walt Disney Studios 2019 Release Schedule Unve...


In [60]:
posts_4['is_tv'] = posts_4['subreddit'].map({'movies': 0, 'television': 1})

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
posts_4.head()

,subreddit,posts_combined,is_tv
0,movies,"Hi, I’m Trevor Stevens. Director of the film, ...",0
1,movies,David Harbour Cast In Scarlett Johansson’s ‘Bl...,0
2,movies,JOKER - Teaser Trailer - In Theaters October 4,0
3,movies,Guillermo del Toro Expands the World of ‘Pan’s...,0
4,movies,Walt Disney Studios 2019 Release Schedule Unve...,0


In [62]:
posts_4.tail()

,subreddit,posts_combined,is_tv
1111,television,Joey King is amazing in “The Act”,1
1112,television,How many streaming services do you subscribe to?,1
1113,television,"Cary Joji Fukunaga (""Maniac"") Explains Why TV ...",1
1114,television,Ryan Murphy’s ‘The Politician’ to Premiere on ...,1
1115,television,How can I watch Game of Thrones season 8 LIVE?,1


In [48]:
posts_1.drop_duplicates(inplace=True)

In [49]:
posts_2.drop_duplicates(inplace=True)

In [63]:
posts_4.drop_duplicates(inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
posts_4.shape

(1113, 3)

In [33]:
#posts_df_3['subreddit'] = posts_df_3['subreddit'].map({'movies': 0, 'television': 1})

In [65]:
posts = [posts_1, posts_2, posts_4]

In [67]:
df = pd.concat(posts, sort=False).reset_index(drop=True)

In [68]:
len(df)

3405

In [70]:
import numpy as np

In [71]:
mask = np.random.rand(len(df)) < 0.6 # generates a random split of the dataframe by 60%

# assigns the split to a train and test (or holdout) set
train = df[mask]
test = df[~mask]

In [72]:
len(train)

2061

In [73]:
len(test)

1344

In [74]:
train.head()

,posts_combined,subreddit,is_tv
0,Official Discussion Weekly Megathread (Pet Sem...,movies,0
2,"Twenty years ago, an upstart animator named Mi...",movies,0
6,10 years later and Avatar still holds the reco...,movies,0
7,‘Hellboy’ Receives Rare R 18+ Rating in Austra...,movies,0
8,"In 2015, Maggie Gyllenhaal, then 37, was told ...",movies,0


In [75]:
test.tail()

,posts_combined,subreddit,is_tv
3395,Doom Patrol 1x07 Promo “Therapy Patrol”,television,1
3396,What are you watching and what do you recommen...,television,1
3397,'Perry Mason' Limited Series starring Matthew ...,television,1
3400,Joey King is amazing in “The Act”,television,1
3404,How can I watch Game of Thrones season 8 LIVE?,television,1


In [78]:
# saves the data as a csv without indices
train.to_csv('train.csv', index=False)

In [79]:
test.to_csv('test.csv', index=False)